# ⌛ Tabellen

* Wir arbeiten häufig mit Tabellen , die z.B. Metadaten enthalten.
* Eine gängige Art der Darstellung von Daten ist eine Text-Datei, in der die Spalten durch Kommas oder einen anderen Delimiter getrennt sind.
* Wir können auch Excel-Dateien verwenden, um Daten zu speichern und zu laden.
* Pandas ist eine gängige  Bibliothek,  um tabellarische Daten zu manipulieren und zu visualisieren.
* Wir schauen uns eine Datei zu den Sprecher:innen im Comigs-Korpus an.
* Die Datei hat drei Spalten: ID, Proficiencly level und set.
  * Es gibt in Comigs zwei Unterkorpora , set 1 und set 2 , die sich u.a. dadurch unterscheiden, dass es beim einen  set zwei manuell erstellte Zielhypothesen gibt und  beim anderen nicht.



In [ ]:
# wir importieren zunächst zwei Pakete, die wir unten benutzen wollen
import json
import pandas as pd


In [ ]:
import pandas as pd
# Wir laden die Datei in einen Pandas-Dataframe mit Hilfe einer Methode der Klasse pandas.
# Der Parameter sep="\t" besagt, dass wir  Tab-getrennte Daten haben. 
# (Wir verwenden deswegen auch als Dateinamenserweiterung .tsv und nicht .csv .)
df = pd.read_csv('./data/comigs_proficiency_levels.tsv',sep="\t")

# Mit head können wir uns die ersten Zeilen des Dataframes anzeigen lassen
print(f"HEAD:\n {df.head()}") 
# Das Gegenstück für die letzten Zeilen ist tail()

print(f"TAIL:\n {df.tail()}") 

# Ausgabe der ganzen Tabelle
print(df)

# Länge der Tabelle
print(f"Tabelle hat {len(df)} Zeilen")

In [ ]:
# Wir können uns Information über die Datentypen in den Spalten anzeigen lassen:
print(df.info())

# Ebenso macht es meist Sinn zu prüfen, ob die Daten vollständig sind.
print("Fehlende Werte je Spalte: \n")
print(df.isnull().sum())



In [ ]:
# Wir sortieren die Tabelle: zuerst aufsteigend nach set, dann nach Proficiency level, und dann nach ID
# inplace=True führt dazu, dass die sortierte Tabelle direkt in df gespeichert wird, ohne einen neuen Dataframe zu erzeugen.
df.sort_values(by=["set", "Proficiency level", "ID" ], ascending=True, inplace=True)  
print(df)


In [ ]:
# Wenn wir nur an Set 1 interessiert sind, können wir  einen Unter-Datenframe erzeugen, in dem wir nur die Zeilen für set 1  behalten 
# NB: beachten Sie die Syntax, mit der  wir die Zeilen des Dataframes filtern.
set1_df = df[ df["set"] == 1 ]
print(set1_df)
print(len(set1_df))

# wir können den neuen Dataframe in eine TSV-Datei speichern - pandas dataframes haben dazu die Methode `to_csv`.
set1_df.to_csv("set1_df.tsv",sep="\t", index=False)

## 🫵 Your turn

* Erstellen Sie eine Liste mit den IDs der Lerner:innen aus Set2, die das GER-Niveau A2 haben (Spalte 'Proficiency level').
* Kommentieren Sie den load-Befehl in der nächsten Zelle aus, wenn Sie eine Lösung sehen wollen.


In [ ]:
# %load ./snippets/find_set2_level_a2_learners.py
set2_ids = df[ df["set"] == 2 ]["ID"].tolist()
a2_ids = df[ df["Proficiency level"] == "A2"]["ID"].tolist()
schnittmenge=set(set2_ids).intersection(set(a2_ids))
print(list(schnittmenge))

# Kompaktere Alternative, die die Funktionalität von pandas nutzt: 
# wir bilden einen neuen Dataframe, in dem wir nur die Zeilen für Set 2 und Proficiency level A2 behalten.
set2_a2_df = df[ (df["set"] == 2) & (df["Proficiency level"] == "A2") ]
print(set2_a2_df["ID"].tolist())



In [ ]:
# Zugriff auf eine einzelne Spalte:
print(df["Proficiency level"])

# Welche Proficiency levels kommen in der Spalte "Proficiency level" vor?
print(df["Proficiency level"].unique())
# Wie oft kommt jeder der Proficiency levels vor?
print(df["Proficiency level"].value_counts())

## 🫵 Your turn

* 	Wie können Sie bestimmen, wie viele Lerner:innen in Set 1 und Set 2 jeweils vorhanden sind?

In [ ]:
# %load ./snippets/count_learners_per_set.py


In [ ]:
# Wir wollen schauen , ob die gleichen Proficiency levels in beiden sets vorkommen.

set1_proficiency_levels = df[df["set"] == 1]["Proficiency level"].unique()
set2_proficiency_levels = df[df["set"] == 2]["Proficiency level"].unique()

print(f"Set 1 Proficiency Levels: {set1_proficiency_levels}")
print(f"Set 2 Proficiency Levels: {set2_proficiency_levels}")


In [ ]:
# Wir möchten uns nun die Anzahl der Lerner:innen pro Proficiency level pro Set anzeigen lassen.
# Wir erstellen mit der Funktion crosstab eine Kreuztabelle aus den beiden Variablen.
print(pd.crosstab(df["Proficiency level"], df["set"]))

In [ ]:
# Wir möchten die Daten in der Tabelle umformatieren und die Info in Proficiency levels in 2 Spalten "Proficiency Min" und "Proficeincy Max" aufteilen.
# Wenn ein einfacher Wert in der Spalte "Proficiency level" steht, schreiben wir ihn in die beiden neuen Spalten "Proficiency Min" und "Proficiency Max" .
# Wenn ein Wert mit "/" in der Spalte steht (z.B. A2/B1), trennen wir den Eintrag und schreiben den linken Wert in "Proficiency Min" und den rechten Wert in "Proficiency Max".

# Wir können neue Spalten zu einem Datenframe hinzufügen.

df["Proficiency Min"]=""
df["Proficiency Max"]=""
for ix, row in df.iterrows():
	if "/" in row["Proficiency level"]:
		df.loc[ix, "Proficiency Min"] = row["Proficiency level"].split("/")[0]
		df.loc[ix, "Proficiency Max"] = row["Proficiency level"].split("/")[1]
	else:
		df.loc[ix, "Proficiency Min"] = row["Proficiency level"]
		df.loc[ix, "Proficiency Max"] = row["Proficiency level"]
print(df.tail())

## 🫵 Your turn

* Verändern Sie den obigen Code so, dass Sie möglichst die Methoden aus dem Modul `re` verwenden, um die Einträge in "Proficiency Min" und "Proficiency Max" zu extrahieren.
* (Sie können eine Lösung sehen, in dem  Sie den load-Befehl in der nächsten Zelle auskommentieren, den Code laden und dann ausführen.

In [ ]:
# %load ./snippets/use_re_module.py



* In der Code-Zelle unten können Sie eine noch eine kompaktere Version sehen, um die Info in Proficiency level aufzuteilen.

In [ ]:
# %load ./snippets/apply_example.py


## 🫵 Your turn

* Erstellen Sie neue Kreuztabellen mit den Proficiency levels in "Proficiency Min" und "Proficiency Max" pro Set.

In [ ]:
# %load ./snippets/create_crosstabs.py
crosstab_min = pd.crosstab(df["Proficiency Min"], df["set"])
crosstab_max = pd.crosstab(df["Proficiency Max"], df["set"])

print(crosstab_min)
print(crosstab_max)


## Erstellen von Dataframes aus Listen  und Dictionaries

* DataFrames / Tabellen haben eine natürliche Verwandtschaft zu Listen und Dictionaries.
* Wir können die einzelnen Spalten als Listen verstehen.
* Wir können Zeilen als Dictionaries verstehen, wobei die Schlüssel die Spaltennamen und die Werte die Spaltenwerte in den relevanten Zellen sind.


In [ ]:
# In einem anderen Notebook  haben wir Worthäufigkeiten für eine Datei aus dem Merlin-Korpus ermittelt
# und in dem Dictionary `freq_records` gespeichert.
# Wir laden diese Info hier wieder
with open("./data/copy_freq_data.json", "r") as f:
    freq_records = json.load(f)



print(freq_records)

# Wir erzeugen aus den Schlüsseln und Werten zwei parallele Listen.
# NB: die keys()-Methode von dictionaries gibt nicht direkt eine Liste zurück.
# Wir können den Rückgabewert mit list() wieder in eine Liste umwandeln.
words = list(freq_records.keys())
freqs = list(freq_records.values())
# Wir schauen uns einige der Werte in den Listen an.
print(words[0:5])	
print(freqs[0:5])
# Erstellen eines Dataframes aus parallelen Listen
# Beachte: wir müssen am Ende ein dictionary haben, dessen Schlüssel die Spaltennamen und dessen Werte die zugehörigen Listen für die Werte in den Zeilen  sind.
df_words_freqs = pd.DataFrame({"word": words, "frequency": freqs})
print(df_words_freqs.head())


In [ ]:

# Wir haben in einem anderen Notebook ein Dictionary namens `records` mit den Häufigkeiten von Bigrammen erzeugt.
#  Das hatten wir auf Disk gespeihert. Wir laden diese Info hier wieder.

with open("./data/copy_of_bigram_records.json", "r") as f:
    records = json.load(f)

print(records)



# Wir wandeln die Information  in eine Liste von Dictionaries um. 
# Jedes dictionary hat 2 key-value Paare, eines für das Bigramm und eines für seine Häufigkeit.
# Aus der Liste von dictionaries können wir einen DataFrame erzeugen.
row_list  =[]
for kee,val in records.items():
	row_dict = {"bigram": kee, "frequency": val}
	row_list.append(row_dict)
print(f"row_list {row_list[0:5]}")
bigram_df = pd.DataFrame(row_list)
print(bigram_df.head())	


## 🫵 Your turn 


* Oben haben wir ein Dictionary `vowel_frequencies` erstellt.
* Wandeln Sie die Werte in einen Dataframe um.


In [ ]:
# zunächst laden wir vowel frequencies wieder

import json

# Load from file
with open("./data/copy_of_vowel_frequencies.sjon", "r") as f:
    vowel_frequencies = json.load(f)

In [ ]:
# %load ./snippets/vowel_frequencies_df.py
# convert the dictionary vowel_frequencies to a data frame with columns "vowel" and "frequency"
print(vowel_frequencies)
row_list = []
for vowel, freq in vowel_frequencies.items():
    row_dict = {"vowel": vowel, "frequency": freq}
    row_list.append(row_dict)
vowel_df = pd.DataFrame(row_list)
print(vowel_df.head())


* 🚀 In der folgenden Zelle sind noch zwei alternative Lösungen, die kompakter sind.

In [ ]:
# %load ./snippets/vowel_frequencies_df_compact.py
# eine weitere noch kompaktere Option
df_vowel_freq = pd.DataFrame.from_records(list(vowel_frequencies.items()),columns=["vowel", "frequency"])
print(df_vowel_freq)


# noch eine  kompakte Option, bei der wir die Vokale als Indizes verwenden statt als WeRte in einer Spalte.
df_vowel_freq = pd.DataFrame.from_dict(vowel_frequencies, orient="index", columns=["frequency"])
print(df_vowel_freq.head())


# Visualisierung

* pandas stellt auch Methoden bereit , um Daten zu visualiseren.
* Zum Beispiel können wir aus den oben erstellten Crosstabs  Barplots erstellen.


In [ ]:
crosstab_min.plot(kind="bar", stacked=True, rot=0)
crosstab_min.plot(kind="bar", stacked=False, rot=0)